In [2]:
import pandas as pd
import numpy as np 

data = pd.read_csv("/home/j-k13b204/S13P31B204/model_test/Code/csv_collection/emotion_grouping.csv")

data.head()

,text,expression,sentiment
0,멀티 채널은 기존에 우리가 사용하고 있는 채널을 말하는데 어 쉽게 이야기하면 티비나...,u-fact,uncertain
1,반면에 옴니 채널은 온 오프라인 모바일 등 다양한 채널을 통해서 상품을 검색하고 구...,u-fact,uncertain
2,예를 들어 스마트폰 근거리 통신 기술을 이용해서 마트 앞을 지나는 고객에게 쿠폰을 ...,u-fact,uncertain
3,두 개에 차이점은 멀티 채널은 운영하고 있는 각 채널을 독립적으로 운영해서 온 오프...,u-fact,uncertain
4,제가 중국에서 일 년 정도 유학 생활을 했었는데요.,u-fact,uncertain


In [ ]:
label_map = {"uncertain": 0, "negative": 1, "positive": 2}

data["label_encoding"] = (
    data["sentiment"]
        .map(label_map)            # 문자열 → 숫자 매핑
        .fillna(-1)                # 혹시 없는 값은 -1 처리 (unknown)
        .astype(int)
)

data['label_encoding'].value_counts()

label_encoding
0    9488
2    8068
1    2231
Name: count, dtype: int64

In [17]:
data[data['label_encoding'] == 1]

,text,expression,sentiment,label_encoding,len
31,어 죽음이 두려운 건 사실입니다.,n-anxiety,negative,1,18
45,한동안 이런 부분 때문에 힘들어서 일 년 휴학도 했지만 계속 혼자 힘으로 이 대학 ...,n-distress,negative,1,63
85,하지만 분명히 그 사람을 도울 이유도 없고 그렇다고 물론 나를 꼭 도와야 될 이유는...,n-sadness,negative,1,134
90,혼자 일을 하는 경우는 감정적으로 고독하다는 것 보다도 내가 모든 것을 결정하고 수...,n-anxiety,negative,1,89
108,그때 틀린 것으로 인해서 대학교의 레벨이 갈릴 수 있는 그런 큰 문제였기 때문에 그...,n-distress,negative,1,66
...,...,...,...,...,...
19687,마음이 너무 약해 남이 아파할 것을 먼저 생각하고 남의 마음을 먼저 생각해다 보니 ...,n-distress,negative,1,104
19691,또 제 나이에서 오는 어떤 신체적인 그런 위험 어떤 불안함도 있을 수 있구요.,n-anxiety,negative,1,45
19712,이 문제를 다른 사람에게 말하는 것부터 어린 시절 소심한 저에게 있어서 부끄럽고 어...,n-shame,negative,1,56
19718,그냥 저는 몰려오는 우울감에 항상 어려움을 겪었습니다.,n-distress,negative,1,30


In [9]:
data['len'] = data['text'].str.len()
print(data['len'].mean())
print(data['len'].max())
print(data['len'].min())

73.53297619649264
506
6


In [12]:
result = (data['len'] >= 75).sum()
result

7534

## 모델 제작

In [1]:
import torch
import torch.nn as nn
from torch.optim import AdamW    
from torch.utils.data import Dataset, DataLoader

import numpy as np
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup 

from kobert_transformers import get_kobert_model, get_tokenizer

# BERT 모델/토크나이저
bertmodel = get_kobert_model()
tokenizer = get_tokenizer()

device = torch.device("cuda")

#### SoftMax + CrossEntropy

In [18]:
# 하이퍼파라미터
NUM_LABELS   = 3   # 0 : uncertain, 1 : negative, 2 : positive
MAX_LEN      = 256
BATCH_SIZE   = 16
EPOCHS       = 10
LR           = 2e-5
WEIGHT_DECAY = 0.01
DROPOUT      = 0.2
PATIENCE     = 2  # validation이 2epoch동안 증가 안 할시 멈춰

In [ ]:
import os, random
import numpy as np

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# MAX_LEN 안전선
assert MAX_LEN <= 256


In [ ]:
import torch
import torch.nn as nn

class KoBERTCLSClassifier(nn.Module):
    def __init__(
        self,
        bert,
        hidden_size: int = 768,
        num_classes: int = 3,           # 0: uncertain, 1: negative, 2: positive
        dropout: float = 0.3,           
        class_weights: torch.Tensor | None = None,
        label_smoothing: float = 0.0,   # 원하면 활성화 가능
        freeze_bert: bool = False       # 초반 동결 여부
    ):
        super().__init__()
        self.bert = bert

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

        self.dropout = nn.Dropout(p=dropout)
        self.classifier = nn.Linear(hidden_size, num_classes)
        
        # 감정 분류용 → CrossEntropyLoss
        self.loss_fn = nn.CrossEntropyLoss(
            weight=class_weights,
            label_smoothing=label_smoothing
        ) if class_weights is not None else nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor | None = None,
        token_type_ids: torch.Tensor | None = None,
        labels: torch.Tensor | None = None,
    ):
        # 1️⃣ KoBERT 인코더 통과
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # 2️⃣ [CLS] 토큰 벡터 사용
        if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
            pooled = outputs.pooler_output             # (B, 768)
        else:
            pooled = outputs[0][:, 0]                  # last_hidden_state[:, 0]

        # 3️⃣ Dropout → Linear
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)               # (B, num_classes)

        # 4️⃣ Loss 계산 (학습 시)
        if labels is not None:
            if labels.dtype != torch.long:
                labels = labels.long()
            loss = self.loss_fn(logits, labels)
            return logits, loss

        return logits, None


In [20]:
data.columns

Index(['text', 'expression', 'sentiment', 'label_encoding', 'len'], dtype='object')

In [22]:
data['label_encoding'].value_counts()

label_encoding
0    9488
2    8068
1    2231
Name: count, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
import numpy as np

train_df, valid_df = train_test_split(
    data.dropna(subset =['text','label_encoding']),
    test_size = 0.3, random_state = 42, stratify=data['label_encoding']
)
counts = train_df['label_encoding'].value_counts().sort_index().to_numpy()
N, K = counts.sum(), counts.shape[0]
class_weights = torch.tensor([float(N/(counts[i]*K)) for i in range(K)], dtype=torch.float32).to(device)
print("class_weights:", class_weights)
print(counts)

class_weights: tensor([0.6952, 2.9556, 0.8175], device='cuda:0')
[6641 1562 5647]


In [24]:
from torch.utils.data import Dataset, DataLoader

# 한 문장씩 KoBert토크나이즈
class IntentDataset(Dataset) : 
    def __init__(self, data, tokenizer, max_len) : 
        self.texts = data['text'].astype(str).tolist()
        self.labels = data['label_encoding'].astype(int).tolist()
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self) : return len(self.texts)

    def __getitem__(self, idx) :
        enc = self.tok(
            self.texts[idx],
            padding = 'max_length',
            truncation = True,
            max_length = self.max_len,
            return_tensors = 'pt'
        )

        item = {
            "input_ids" : enc["input_ids"].squeeze(0),
            "attention_mask" : enc["attention_mask"].squeeze(0),
            "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
        }

        if "token_type_ids" in enc : 
            item["token_type_ids"] = enc["token_type_ids"].squeeze(0)
        else : 
            item['token_type_ids'] = None
        return item

# 미니배치 단위로 텐서 묶음
train_loader = DataLoader(IntentDataset(train_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
valid_loader = DataLoader(IntentDataset(valid_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [26]:
# 모델 옵티마이저, 스케쥴러 세팅

from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup

model = KoBERTCLSClassifier(
    bert = bertmodel,
    hidden_size=768,
    num_classes = NUM_LABELS,
    dropout=DROPOUT,
    class_weights=class_weights
).to(device)

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

total_steps = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * 0.06)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

In [ ]:
# === 학습/검증 루프 (감정 3클래스) ===
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import os

USE_AMP = torch.cuda.is_available()
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP)

LABEL2ID = {'uncertain': 0, 'negative': 1, 'positive': 2}
ID2LABEL = {v:k for k,v in LABEL2ID.items()}
label_names = [ID2LABEL[i] for i in range(3)]  # ['uncertain','negative','positive']

def _to_device(batch, device):
    input_ids      = batch["input_ids"].to(device, non_blocking=True)
    attention_mask = batch["attention_mask"].to(device, non_blocking=True)
    token_type_ids = batch.get("token_type_ids", None)
    if token_type_ids is None:
        token_type_ids = torch.zeros_like(attention_mask)
    token_type_ids = token_type_ids.to(device, non_blocking=True)
    labels         = batch["labels"].to(device, non_blocking=True)
    return input_ids, attention_mask, token_type_ids, labels

def train_one_epoch(model, loader):
    model.train()
    total_loss, all_p, all_y = 0.0, [], []
    for batch in tqdm(loader, desc="Train"):
        input_ids, attention_mask, token_type_ids, labels = _to_device(batch, device)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=USE_AMP):
            logits, loss = model(input_ids, attention_mask, token_type_ids, labels)

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer); scaler.update()
        scheduler.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        all_p.extend(preds.detach().cpu().tolist())
        all_y.extend(labels.detach().cpu().tolist())

    denom = max(1, len(loader))
    return total_loss/denom, accuracy_score(all_y, all_p), f1_score(all_y, all_p, average="macro")

@torch.no_grad()
def evaluate(model, loader, target_names=None):
    model.eval()
    total_loss, all_p, all_y = 0.0, [], []
    for batch in tqdm(loader, desc="Valid"):
        input_ids, attention_mask, token_type_ids, labels = _to_device(batch, device)
        with torch.cuda.amp.autocast(enabled=USE_AMP):
            logits, loss = model(input_ids, attention_mask, token_type_ids, labels)
        if loss is not None:
            total_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        all_p.extend(preds.detach().cpu().tolist())
        all_y.extend(labels.detach().cpu().tolist())

    denom = max(1, len(loader))
    rep = classification_report(
        all_y, all_p,
        labels=[0,1,2],                  # 순서 고정
        target_names=target_names,       # ['uncertain','negative','positive']
        digits=4,
        zero_division=0
    )
    return total_loss/denom, accuracy_score(all_y, all_p), f1_score(all_y, all_p, average="macro"), rep

# === 학습 실행 ===
best_f1, patience = 0.0, 0
save_dir = "./kobert_sentiment_model"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(1, EPOCHS + 1):
    print(f"\n==== EPOCH {epoch}/{EPOCHS} ====")
    tr_loss, tr_acc, tr_f1 = train_one_epoch(model, train_loader)
    val_loss, val_acc, val_f1, val_rep = evaluate(model, valid_loader, target_names=label_names)

    print(f"[Train] loss={tr_loss:.4f} acc={tr_acc:.4f} f1={tr_f1:.4f}")
    print(f"[Valid] loss={val_loss:.4f} acc={val_acc:.4f} f1={val_f1:.4f}")
    print(val_rep)

    if val_f1 > best_f1:
        best_f1, patience = val_f1, 0
        torch.save(model.state_dict(), os.path.join(save_dir, "model.pt"))
        with open(os.path.join(save_dir, "label_map.txt"), "w", encoding="utf-8") as f:
            for i, name in enumerate(label_names):   # 0→uncertain, 1→negative, 2→positive
                f.write(f"{i}\t{name}\n")
        print(f">>> Best saved. macroF1={best_f1:.4f}")
    else:
        patience += 1
        if patience >= PATIENCE:
            print("Early stopping.")
            break



==== EPOCH 1/10 ====


Valid: 100%|█████████████████| 372/372 [00:14<00:00, 25.51it/s]


[Train] loss=0.3717 acc=0.9581 f1=0.9566
[Valid] loss=0.5211 acc=0.9469 f1=0.9455
              precision    recall  f1-score   support

   uncertain     0.9429    0.9505    0.9467      2847
    negative     0.9654    0.9178    0.9410       669
    positive     0.9469    0.9508    0.9489      2421

    accuracy                         0.9469      5937
   macro avg     0.9517    0.9397    0.9455      5937
weighted avg     0.9471    0.9469    0.9469      5937

>>> Best saved. macroF1=0.9455

==== EPOCH 2/10 ====


Valid: 100%|█████████████████| 372/372 [00:14<00:00, 25.70it/s]


[Train] loss=0.3109 acc=0.9674 f1=0.9666
[Valid] loss=0.5118 acc=0.9446 f1=0.9442
              precision    recall  f1-score   support

   uncertain     0.9661    0.9203    0.9426      2847
    negative     0.9264    0.9596    0.9427       669
    positive     0.9265    0.9690    0.9473      2421

    accuracy                         0.9446      5937
   macro avg     0.9397    0.9496    0.9442      5937
weighted avg     0.9455    0.9446    0.9445      5937


==== EPOCH 3/10 ====


...

[Train] loss=0.1298 acc=0.9873 f1=0.9874
[Valid] loss=0.5841 acc=0.9506 f1=0.9501
              precision    recall  f1-score   support

   uncertain     0.9559    0.9431    0.9494      2847
    negative     0.9437    0.9522    0.9479       669
    positive     0.9466    0.9591    0.9528      2421

    accuracy                         0.9506      5937
   macro avg     0.9487    0.9515    0.9501      5937
weighted avg     0.9507    0.9506    0.9506      5937

>>> Best saved. macroF1=0.9501

==== EPOCH 5/10 ====


Valid: 100%|█████████████████| 372/372 [00:14<00:00, 26.21it/s]


[Train] loss=0.0860 acc=0.9916 f1=0.9914
[Valid] loss=0.6343 acc=0.9510 f1=0.9490
              precision    recall  f1-score   support

   uncertain     0.9489    0.9526    0.9507      2847
    negative     0.9392    0.9462    0.9427       669
    positive     0.9568    0.9504    0.9536      2421

    accuracy                         0.9510      5937
   macro avg     0.9483    0.9497    0.9490      5937
weighted avg     0.9510    0.9510    0.9510      5937


==== EPOCH 6/10 ====


Valid: 100%|█████████████████| 372/372 [00:14<00:00, 25.44it/s]

[Train] loss=0.0616 acc=0.9936 f1=0.9936
[Valid] loss=0.5865 acc=0.9491 f1=0.9474
              precision    recall  f1-score   support

   uncertain     0.9617    0.9350    0.9482      2847
    negative     0.9201    0.9641    0.9416       669
    positive     0.9433    0.9616    0.9523      2421

    accuracy                         0.9491      5937
   macro avg     0.9417    0.9536    0.9474      5937
weighted avg     0.9495    0.9491    0.9491      5937

Early stopping.


In [12]:
# ==== Inference: Load saved model and predict per sentence ====
import re, torch
import torch.nn as nn
from kobert_transformers import get_kobert_model, get_tokenizer

# ---- 1) 하이퍼파라미터 & 라벨 매핑 ----
DEVICE  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LEN = 256
LABEL2ID = {'uncertain': 0, 'negative': 1, 'positive': 2}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

# ---- 2) 모델 클래스 (학습 때 쓴 것과 동일) ----
class KoBERTCLSClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dropout=0.3,
                 class_weights=None, label_smoothing=0.0, freeze_bert=False):
        super().__init__()
        self.bert = bert
        if freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False
        self.dropout = nn.Dropout(p=dropout)
        self.classifier = nn.Linear(hidden_size, num_classes)
        # 추론에서는 loss를 쓰지 않지만, 키 불일치 방지를 위해 속성은 유지
        self.loss_fn = nn.CrossEntropyLoss(
            weight=class_weights,
            label_smoothing=label_smoothing
        ) if class_weights is not None else nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled = outputs.pooler_output if getattr(outputs, "pooler_output", None) is not None \
                 else outputs[0][:, 0]  # [CLS]
        logits = self.classifier(self.dropout(pooled))
        if labels is not None:
            if labels.dtype != torch.long:
                labels = labels.long()
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None

# ---- 3) 모델/토크나이저 로드 & 가중치 불러오기 ----
bert = get_kobert_model()
tokenizer = get_tokenizer()

model = KoBERTCLSClassifier(
    bert=bert,
    hidden_size=768,
    num_classes=3,
    dropout=0.2  # 학습 시 사용값과 맞추면 가장 좋음
).to(DEVICE)

ckpt_path = "/home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/model.pt"
state = torch.load(ckpt_path, map_location=DEVICE)
_ = model.load_state_dict(state, strict=False)  # <- 핵심: loss_fn.weight 등 키 불일치 무시
model.eval()

# ---- 4) 문장 단위 분할 함수 ----
def split_sentences(paragraph: str):
    # 줄 단위 → 종결부호(., ?, !) 기준 추가 분할, 한글 종결('다.', '요.') 보강
    lines = [s.strip() for s in paragraph.strip().splitlines() if s.strip()]
    sents = []
    for line in lines:
        pieces = re.split(r'(?<=[\.!?])\s+|(?<=다\.)\s+|(?<=요\.)\s+', line)
        sents += [p.strip() for p in pieces if p and p.strip()]
    return sents

# ---- 5) 예측 함수 ----
@torch.no_grad()
def predict_sentences(sent_list):
    results, probs_all = [], []
    for sent in sent_list:
        enc = tokenizer(
            sent,
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt",
            return_token_type_ids=True
        )
        input_ids      = enc["input_ids"].to(DEVICE)
        attention_mask = enc["attention_mask"].to(DEVICE)
        token_type_ids = enc.get("token_type_ids", torch.zeros_like(attention_mask)).to(DEVICE)

        logits, _ = model(input_ids, attention_mask, token_type_ids, labels=None)
        prob = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()  # [3]
        pred_id = int(prob.argmax())
        results.append({
            "text": sent,
            "pred_id": pred_id,
            "pred_label": ID2LABEL[pred_id],
            "probs": {
                "uncertain": float(prob[LABEL2ID['uncertain']]),
                "negative":  float(prob[LABEL2ID['negative']]),
                "positive":  float(prob[LABEL2ID['positive']]),
            }
        })
        probs_all.append(prob)

    # 문단 전체 스코어(평균 확률 기반)
    whole_pred = None
    if probs_all:
        import numpy as np
        mean_prob = np.stack(probs_all, axis=0).mean(axis=0)
        whole_pred = {
            "paragraph_pred_id": int(mean_prob.argmax()),
            "paragraph_pred_label": ID2LABEL[int(mean_prob.argmax())],
            "paragraph_probs": {
                "uncertain": float(mean_prob[LABEL2ID['uncertain']]),
                "negative":  float(mean_prob[LABEL2ID['negative']]),
                "positive":  float(mean_prob[LABEL2ID['positive']]),
            }
        }
    return results, whole_pred

# ---- 6) 예측 실행 ----
paragraph = """
저는 데이터를 통해 더 나은 판단을 만들 수 있다고 믿습니다. SSAFY 교육 과정에서 Java, Python을 기반으로 웹·데이터 분석 프로젝트를 수행하며 문제를 구조적으로 바라보는 습관을 기르게 되었습니다.
특히 “성남시 공영주차장 입지 분석” 프로젝트에서는 실제 행정 데이터를 활용해 선형회귀 모델을 구축하고, 상권 밀집도·불법주정차 건수 등 변수를 조합해 정책 관점의 시나리오 분석을 진행했습니다.

이 경험을 통해 단순히 모델을 만드는 것을 넘어, 데이터를 기반으로 실제 이해관계자가 의사결정을 내릴 수 있도록 해석하는 능력을 갖추게 되었습니다. 앞으로도 데이터를 통해 문제를 명확하게 정의하고, 구조화된 방식으로 해결하는 데이터 분석가가 되고 싶습니다.
"""

sents = split_sentences(paragraph)
sent_results, para_result = predict_sentences(sents)

print("=== Sentence-level ===")
for r in sent_results:
    print(f"- {r['text']}")
    print(f"  -> pred: {r['pred_label']}  probs={r['probs']}")

print("\n=== Paragraph-level (mean of probs) ===")
print(para_result)


=== Sentence-level ===
- 저는 데이터를 통해 더 나은 판단을 만들 수 있다고 믿습니다.
  -> pred: uncertain  probs={'uncertain': 0.9999991655349731, 'negative': 1.0574576236876965e-07, 'positive': 7.207909789030964e-07}
- SSAFY 교육 과정에서 Java, Python을 기반으로 웹·데이터 분석 프로젝트를 수행하며 문제를 구조적으로 바라보는 습관을 기르게 되었습니다.
  -> pred: uncertain  probs={'uncertain': 0.9999980926513672, 'negative': 9.087739272217732e-08, 'positive': 1.75665013557591e-06}
- 특히 “성남시 공영주차장 입지 분석” 프로젝트에서는 실제 행정 데이터를 활용해 선형회귀 모델을 구축하고, 상권 밀집도·불법주정차 건수 등 변수를 조합해 정책 관점의 시나리오 분석을 진행했습니다.
  -> pred: uncertain  probs={'uncertain': 0.999998927116394, 'negative': 1.1433208868538713e-07, 'positive': 1.0087101145472843e-06}
- 이 경험을 통해 단순히 모델을 만드는 것을 넘어, 데이터를 기반으로 실제 이해관계자가 의사결정을 내릴 수 있도록 해석하는 능력을 갖추게 되었습니다.
  -> pred: uncertain  probs={'uncertain': 0.9999983310699463, 'negative': 9.653924593067131e-08, 'positive': 1.5832737290111254e-06}
- 앞으로도 데이터를 통해 문제를 명확하게 정의하고, 구조화된 방식으로 해결하는 데이터 분석가가 되고 싶습니다.
  -> pred: positive  probs={'uncertain': 3.0188539312803186e-07, '

In [ ]:
# quantize_kobert.py
import os
import torch
import torch.nn as nn
import torch.ao.quantization as quant
from kobert_transformers import get_kobert_model

# ===== 0) 사용자 환경 =====
MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/model.pt"  # 네가 준 경로
OUTPUT_DIR = os.path.join(os.path.dirname(MODEL_PATH), "quantize")
os.makedirs(OUTPUT_DIR, exist_ok=True)
QUANTIZED_PT_PATH = os.path.join(OUTPUT_DIR, "model_quantized.pt")

# ===== 1) 분류기 정의 (학습 때 쓰던 구조와 동일해야 함) =====
class BertClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=0.3, class_weights=None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        # 학습용 손실함수 키가 state_dict에 섞여있을 수 있어 정의만 남김
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled = out.pooler_output if getattr(out, "pooler_output", None) is not None else out[0][:, 0]
        logits = self.classifier(self.dropout(pooled))
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None

def load_fp32_model(model_path: str, num_classes: int = 3, dr_rate: float = 0.3) -> nn.Module:
    device = torch.device("cpu")  # 동적 양자화는 CPU 대상
    print("[1/3] 모델 구조 초기화…")
    bert = get_kobert_model()
    model = BertClassifier(bert=bert, hidden_size=768, num_classes=num_classes, dr_rate=dr_rate).to(device)

    print("[2/3] 가중치 로드…")
    state = torch.load(model_path, map_location=device)

    # state_dict에 학습용 키(예: 'loss_fn.weight')가 섞여 있을 수 있으니 제거
    for k in list(state.keys()):
        if k.startswith("loss_fn"):
            del state[k]

    model.load_state_dict(state, strict=False)
    model.eval()
    print("[3/3] 로드 완료.")
    return model

def quantize_dynamic_int8(model: nn.Module) -> nn.Module:
    print("\n=> 동적 양자화(INT8) 진행 중… (nn.Linear 대상)")
    qmodel = quant.quantize_dynamic(
        model,
        {nn.Linear},        # Linear 층만 INT8
        dtype=torch.qint8   # 가중치 정수화
    )
    print("   완료.")
    return qmodel

if __name__ == "__main__":
    # 필요 시 여기만 바꿔: 감정 모델이면 num_classes=3(예: 부정/중립/긍정)
    NUM_CLASSES = 3

    # 1) FP32 모델 로드
    model = load_fp32_model(MODEL_PATH, num_classes=NUM_CLASSES, dr_rate=0.3)

    # 2) INT8 동적 양자화
    qmodel = quantize_dynamic_int8(model)

    # 3) 저장 (모듈 통째 저장이 재로딩에 가장 안전)
    torch.save(qmodel, QUANTIZED_PT_PATH)

    # 4) 사이즈 출력
    def size_mb(p): return os.path.getsize(p) / (1024 ** 2)
    orig_mb = size_mb(MODEL_PATH)
    quant_mb = size_mb(QUANTIZED_PT_PATH)
    print(f"\n원본 모델:      {orig_mb:.2f} MB")
    print(f"양자화 모델(.pt): {quant_mb:.2f} MB")
    if orig_mb > 0:
        print(f"압축률: {orig_mb/quant_mb:.2f}x, 크기 감소: {(orig_mb-quant_mb)/orig_mb*100:.1f}%")

    print("\n✅ 완료!  (CPU 추론에서 메모리/지연시간 절감 효과 기대)")


In [ ]:
# quantize_kobert_sentiment.py
import os
import torch
import torch.nn as nn
import torch.ao.quantization as quant
from kobert_transformers import get_kobert_model

# ===== 0) 사용자 환경 =====
MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/model.pt"
SAVE_DIR = "/home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/quantize"
os.makedirs(SAVE_DIR, exist_ok=True)
QUANTIZED_PT_PATH = os.path.join(SAVE_DIR, "model_quantized.pt")

# ===== 1) 모델 클래스 정의 =====
class BertClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=0.3, class_weights=None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled = out.pooler_output if getattr(out, "pooler_output", None) is not None else out[0][:, 0]
        logits = self.classifier(self.dropout(pooled))
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None

# ===== 2) 모델 로드 함수 =====
def load_fp32_model(model_path: str, num_classes: int = 3, dr_rate: float = 0.3) -> nn.Module:
    device = torch.device("cpu")
    print("[1/3] 모델 구조 초기화…")
    bert = get_kobert_model()
    model = BertClassifier(bert=bert, hidden_size=768, num_classes=num_classes, dr_rate=dr_rate).to(device)

    print("[2/3] 가중치 로드…")
    state = torch.load(model_path, map_location=device)
    for k in list(state.keys()):
        if k.startswith("loss_fn"):
            del state[k]
    model.load_state_dict(state, strict=False)
    model.eval()
    print("[3/3] 로드 완료.")
    return model

# ===== 3) 양자화 함수 =====
def quantize_dynamic_int8(model: nn.Module) -> nn.Module:
    print("\n=> 동적 양자화(INT8) 수행 중…")
    qmodel = quant.quantize_dynamic(
        model,
        {nn.Linear},
        dtype=torch.qint8
    )
    print("완료.")
    return qmodel

# ===== 4) 실행부 =====
if __name__ == "__main__":
    NUM_CLASSES = 3  # 감정 3분류 (부정/중립/긍정)

    model = load_fp32_model(MODEL_PATH, num_classes=NUM_CLASSES, dr_rate=0.3)
    qmodel = quantize_dynamic_int8(model)

    # 저장
    torch.save(qmodel, QUANTIZED_PT_PATH)
    print(f"\n✅ 양자화된 모델 저장 완료: {QUANTIZED_PT_PATH}")

    # 크기 비교
    def size_mb(path): return os.path.getsize(path) / (1024**2)
    orig = size_mb(MODEL_PATH)
    quantized = size_mb(QUANTIZED_PT_PATH)
    print(f"\n원본 모델 크기: {orig:.2f} MB")
    print(f"양자화 모델 크기: {quantized:.2f} MB")
    print(f"압축률: {orig/quantized:.2f}x, 크기 감소: {(orig - quantized)/orig * 100:.1f}%")


[1/3] 모델 구조 초기화…
[2/3] 가중치 로드…
[3/3] 로드 완료.

=> 동적 양자화(INT8) 수행 중…
   완료.

✅ 양자화된 모델 저장 완료: /home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/quantize/model_quantized.pt

원본 모델 크기: 351.74 MB
양자화 모델 크기: 107.13 MB
압축률: 3.28x, 크기 감소: 69.5%


In [26]:
# infer_quantized_kobert_sentiment.py
# - 동적 양자화(.pt) 모델 로드하여 문장 단위 감정 추론
# - CPU 전용 (quantized model은 CPU에서 효과적)

import re, numpy as np, torch
from kobert_transformers import get_tokenizer
import torch.nn as nn

# ===== 0) 환경/라벨 =====
DEVICE  = torch.device("cpu")   # 동적 양자화 모델은 CPU 추론 권장
MAX_LEN = 256

LABEL2ID = {'uncertain': 0, 'negative': 1, 'positive': 2}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/quantize/model_quantized.pt"

class BertClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=0.3, class_weights=None):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights) if class_weights is not None else nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled = out.pooler_output if getattr(out, "pooler_output", None) is not None else out[0][:, 0]
        logits = self.classifier(self.dropout(pooled))
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return logits, loss
        return logits, None
# ===== 1) 문장 분할 =====
def split_sentences(paragraph: str):
    lines = [s.strip() for s in paragraph.strip().splitlines() if s.strip()]
    sents = []
    for line in lines:
        pieces = re.split(r'(?<=[\.!?])\s+|(?<=다\.)\s+|(?<=요\.)\s+|(?<=,)\s+', line)
        sents += [p.strip() for p in pieces if p and p.strip()]
    return sents

# ===== 2) 모델/토크나이저 로드 =====
print("[Load] tokenizer …")
tokenizer = get_tokenizer()

print(f"[Load] quantized model from: {MODEL_PATH}")
# 전체 모듈로 저장한 양자화 모델은 torch.load로 바로 불러와 사용
model = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)
model.eval()

# ===== 3) 예측 함수 =====
@torch.no_grad()
def predict_sentences(sent_list):
    results, probs_all = [], []
    for sent in sent_list:
        enc = tokenizer(
            sent,
            padding="max_length",
            truncation=True,
            max_length=MAX_LEN,
            return_tensors="pt",
            return_token_type_ids=True
        )
        input_ids      = enc["input_ids"].to(DEVICE)
        attention_mask = enc["attention_mask"].to(DEVICE)
        token_type_ids = enc.get("token_type_ids", attention_mask.new_zeros(attention_mask.size())).to(DEVICE)

        out = model(input_ids, attention_mask, token_type_ids)
        # 양자화 모델의 forward가 (logits, loss) 또는 logits 하나만 반환할 수 있으니 안전 처리
        logits = out[0] if isinstance(out, (tuple, list)) else out

        prob = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()  # [num_classes]
        pred_id = int(prob.argmax())
        results.append({
            "text": sent,
            "pred_id": pred_id,
            "pred_label": ID2LABEL[pred_id],
            "probs": {
                "uncertain": float(prob[LABEL2ID['uncertain']]),
                "negative":  float(prob[LABEL2ID['negative']]),
                "positive":  float(prob[LABEL2ID['positive']]),
            }
        })
        probs_all.append(prob)

    para_summary = None
    if probs_all:
        mean_prob = np.stack(probs_all, axis=0).mean(axis=0)
        para_summary = {
            "paragraph_pred_id": int(mean_prob.argmax()),
            "paragraph_pred_label": ID2LABEL[int(mean_prob.argmax())],
            "paragraph_probs": {
                "uncertain": float(mean_prob[LABEL2ID['uncertain']]),
                "negative":  float(mean_prob[LABEL2ID['negative']]),
                "positive":  float(mean_prob[LABEL2ID['positive']]),
            }
        }
    return results, para_summary

# ===== 4) 테스트 실행 =====
if __name__ == "__main__":
    paragraph = """
계속 실패하면서 스스로가 싫어졌습니다."""
    sents = split_sentences(paragraph)
    sent_results, para_result = predict_sentences(sents)

    print("=== Sentence-level ===")
    for r in sent_results:
        print(f"- {r['text']}")
        print(f"  -> pred: {r['pred_label']}  probs={r['probs']}")

    print("\n=== Paragraph-level (mean of probs) ===")
    print(para_result)

[Load] tokenizer …
[Load] quantized model from: /home/j-k13b204/S13P31B204/model_test/kobert_sentiment_model/quantize/model_quantized.pt
=== Sentence-level ===
- 계속 실패하면서 스스로가 싫어졌습니다.
  -> pred: negative  probs={'uncertain': 3.248096982133575e-05, 'negative': 0.9999674558639526, 'positive': 1.708578736270283e-07}

=== Paragraph-level (mean of probs) ===
{'paragraph_pred_id': 1, 'paragraph_pred_label': 'negative', 'paragraph_probs': {'uncertain': 3.248096982133575e-05, 'negative': 0.9999674558639526, 'positive': 1.708578736270283e-07}}
